In [1]:
from src.utils_.elastic_db import ElasticDB

# INIT DB OBJECT
PORT = "http://localhost:9200"
db = ElasticDB(elastic_port=PORT)

INFO:src.utils_.elastic_db:Connecting to http://localhost:9200 
INFO:src.utils_.elastic_db:Connected to <Elasticsearch(['http://localhost:9200'])> 


In [2]:
### LOAD DATASETS ###
import json
import random

args = [json.loads(ln) for ln in open("../data/cmv_processed.jsonl")]
topics = [json.loads(ln) for ln in open("../data/argument_topic_concept.jsonl")]
concepts = [json.loads(ln) for ln in open("../data/argument_concept.jsonl")]
#extracts = [json.loads(ln) for ln in open("../data/argument_extracts.jsonl")]

In [3]:
len(topics), len(args)

(5990, 10303)

In [4]:
### CHECK BLANKS ###
args_ = [json.loads(ln)["argument"]["argument"] for ln in open("../data/cmv_processed.jsonl")]
ids = [json.loads(ln)["id"] for ln in open("../data/cmv_processed.jsonl")]

for j, k in zip(args_, ids):
    if j == "":
        print("blanks", j, k)

blanks  t3_3cm6jy
blanks  t3_1egv4k
blanks  t3_1egv4k
blanks  t3_5wjdve


In [5]:
unique = set()
idx = set()

for j, k in zip(args_, ids):
    unique.add((j, k))

unique = list(unique)
type(unique), len(unique)

unique

[('Theres always a bully no matter where you go. Theses are basically people who are either physically or verbally abusive. They have a need to showoff their dominance or they have a lot of aggression pentup inside or are psychotic could be anything I guess. The fact is that these guys pick up their victims in a group of people. And they choose the weak guy.What do we mean by a weak guy?Who could be weak in their eyes?',
  't3_6dafc2'),
 ('Its not difficult to understand that the suns ultraviolet rays are dangerous. Today its almost impossible to walk through a supermarket without brushing by shelves full of sun protection products. Nearly every doctors office offers pamphlets on how to check your skin for cancerous lesions and the media repeatedly shares tragic stories about young cancer patients dying because of their tanning obsession. The knowledge is present and its available in front of our eyessunscreen helps protect human skin from these obviously dangerous UV rays.Whats more s

In [6]:
### SUBJECT ARG ###
import random
sample = random.randint(0, 99)

arg = args[sample]["argument"]["argument"]
claim = args[sample]["claim"]
#topic = topics[sample]

print(sample, "\n")
print(claim, "\n")
print(arg, "\n")

76 

Im fairly ashamed of this so please help me . I believe that the vast majority of parents no matter how tolerant and loving would not choose to have a gay child. 

Maybe I shouldnt be making claims about what the vast majority of parents believe so lets just say I believe I would not want to have a gay child if I could actually control that. I do not hate gay people and there are several gay people whom I personally admire. Them being gay doesnt bother me. But when it comes to my own unborn children... I wouldnt want them to be gay.Consider this thought experimentWhen a child is born the parent may select each and everyone one of his or her genetic features. 



In [7]:
### TODOs ###

# IN-PROGRESS: News Data
# TODOs: Add Concepts
# TODOs: Commonsense Query and Concept Expansion: Topics, Concepts, Synonyms
# DONE: Cosine Semantic Search
# TODOs: Research: Evidence Retrieval, Info Retrieval, Context Aware, Neural Retrieval
# TODOs: Targeted Retrieval with NLI over ADUs, Premises, Claims; discard non-ADUs.
# TODOs: Parallel process
# TODOs: Prior tokenization and sentence segmentation to speed processing
# TODOs: Domain Restrict. Polarising social and political debate (Class labelling) only for higher-quality argument-knowledge set.
# TODOs: News, Political, Sociology and 'Good', 'Positive' counter-evidence Knowledge Base.
# TODOs: Bag of Topics Modelling
# TODOs: Implement as a Class
# TODOs: Implement Logging

# TODOs: Implement Semantic Search: https://www.elastic.co/blog/text-similarity-search-with-vectors-in-elasticsearch
# TODOs: Implement Semantic Ranking

In [8]:
### NLP FUNCTIONS ###
import re
from nltk.tokenize import sent_tokenize, word_tokenize

def tokeniser(doc):
    return word_tokenize(doc)

def sentences_segment(doc):
    return sent_tokenize(doc)

# Test Statements
print(tokeniser("hello, my name is Josh!"))
print(sentences_segment("hello, my name is Josh! How are you doing today? I'm curious ... will this line seperate? I'm not so sure Dr. Evil"))

['hello', ',', 'my', 'name', 'is', 'Josh', '!']
['hello, my name is Josh!', 'How are you doing today?', "I'm curious ... will this line seperate?", "I'm not so sure Dr.", 'Evil']


In [9]:
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer

# TODOs: Fix Vectorizer Issue
kb = KeyBERT()
vectorizer = KeyphraseCountVectorizer()
vectorizer_nouns = KeyphraseCountVectorizer(spacy_pipeline="<N. *>")
def extract_keyphrase(doc, n_gram=3, n_kp=3, use_mmr="False", use_maxsum="False"):
    try:
        kp = kb.extract_keywords(doc, keyphrase_ngram_range=(0, 3), stop_words="english", diversity=0.3,)
        kp_ = kb.extract_keywords(doc, vectorizer=vectorizer, stop_words="english", diversity=0.3)

    except:
        return [" "]

    # Concatonate, remove duplicates
    kp = kp + kp_
    kp = [i[0] for i in kp]
    kp = list(set(kp))

    return kp

test = "Brazil's minimum income has increasingly been accepted."
evidence = " "
ev_kp = extract_keyphrase(evidence)
res = [i[0] for i in ev_kp]
print(res)

res_ = res[0]

extract_keyphrase(test)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


[' ']


['income increasingly accepted',
 'minimum income increasingly',
 'brazil',
 'brazil minimum income',
 'brazil minimum',
 'minimum income']

In [10]:
 # def get_topic(arg_id):
#     topic_id = topic_ids.index(arg_id)
#     topic = topics[topic_id]["topic_label"]
#     return str(topic) if topic else None
#
# def get_concept(arg_id):
#     concept_id = concept_ids.index(arg_id)
#     concept = concepts[concept_id]["concept_label"]
#     return str(concept) if concept else None

In [11]:
# TODOs: Manage Duplicate Keywords
# DONE: Sentential Ranking
# DONE: Include Topic Label
# DONE: Include Concept Label
# DONE: Add News
# TODOs: Targeted Retreival with Semantic Graphs
# TODOs: Target Argumentative Content Only
# TODOs: Targeted Argument Content: Adus + Extractive Summary
# TODOs: Query Expansion
# TODOs: Multi-Field Search
# TODOs: Additional News and Knowledge Sources

In [ ]:
from tqdm.notebook import tqdm
from src.detection.stance_classifier import sentence_stance, compare_stance
from src.utils_.word_net_expansion import expand_query
from src.detection.stance_classifier import sentence_stance
import multiprocessing
import json
import time

topic_ids = [json.loads(ln)["id"] for ln in open("../data/argument_topic_concept.jsonl")]
concept_ids = [json.loads(ln)["id"] for ln in open("../data/argument_concept.jsonl")]

def get_notion(notions_ids, notions_lst, arg_id, label):
    notion_id = notions_ids.index(arg_id)
    notion = notions_lst[notion_id][label]
    return str(notion) if notion else None

def extract_adus(arg_):
    arg, id_ = arg_
    print("\n", id_)

    topic = get_notion(topic_ids, topics, id_, "topic_label")
    concept = get_notion(concept_ids, concepts, id_, "concept_label")

    adu_sents = sentences_segment(arg)

    adus = []
    for _ in adu_sents:
        if len(tokeniser(_)) <= 8:
            continue

        kp = extract_keyphrase(_)
        kp.append(topic) if topic else kp
        kp.append(concept) if concept else kp
        print(kp)

        adu = {"sentence": _, "kp": [i for i in kp], "stance": sentence_stance(_, kp[0])}

        adus.append(adu)

    yield ({
        "id": id_,
        "argument": [i for i in adus]
    })

step = max(int(len(unique) / 10), 1)
batches = [unique[i:i + step] for i in range(0, len(unique), step)]

mined_args = []
# TODOs: Remove Huggingface Warnings

for idx, batch in enumerate(batches):
    print('-' * 25 + 'Batch %d/%d' % (idx + 1, len(batches)) + '-' * 25)

    with multiprocessing.Pool(8) as pool:
        with tqdm(total=(len(batch))) as pbar:
            for arg in batch:
                mined_args.append([i for i in extract_adus(arg)])
                pbar.update()

-------------------------Batch 1/10-------------------------
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKEN

  0%|          | 0/599 [00:00<?, ?it/s]


 t3_6dafc2
['theres bully matter', 'bully', 'bully matter', 'theres bully', 'theres', 'the weak', 'weak will power']
['theses basically people', 'physically verbally abusive', 'abusive', 'people', 'verbally abusive', 'theses', 'people physically verbally', 'the weak', 'weak will power']
['dominance lot aggression', 'aggression', 'lot aggression pentup', 'lot aggression', 'need', 'lot', 'dominance', 'anything', 'aggression pentup inside', 'the weak', 'weak will power']
['pick victims', 'fact', 'victims group people', 'people', 'guys pick victims', 'victims', 'guys', 'group', 'pick victims group', 'the weak', 'weak will power']
['weak guy', 'choose weak guy', 'eyes', 'guy mean weak', 'weak guy mean', 'weak guy weak', 'the weak', 'weak will power']

 t3_1xd4gy
['suns ultraviolet rays', 'suns', 'suns ultraviolet', 'ultraviolet rays', 'ultraviolet rays dangerous', 'understand suns ultraviolet', 'skin care', 'uv rays of the sun']
['shelves', 'today impossible', 'brushing shelves sun', 'walk

In [46]:
mined_args[100]

<generator object extract_adus at 0x2c201cc80>

In [20]:
from tqdm.notebook import tqdm
from src.detection.stance_classifier import sentence_stance, compare_stance
from src.utils_.word_net_expansion import expand_query
from src.detection.stance_classifier import sentence_stance
# from multiprocessing.pool import ThreadPool as Pool
import time

### RETRIEVER ###
db = db
queries = []
retrieved_ev = []

topic_ids = [json.loads(ln)["id"] for ln in open("../data/argument_topic_concept.jsonl")]
concept_ids = [json.loads(ln)["id"] for ln in open("../data/argument_concept.jsonl")]

# TODOs: Decouple Argument Mining from Retrieval
# EXPERIMENT: Full Argument vs Extract
def search(mined, retrieve_len=5):
    """ Retrieves Evidence from Knowledge base, returning a well-formed Retrieved Evidence Object
    given an input Argument"""

    id_ = mined["id"]
    print("\n", id_)

    retrieved = []
    for adu in mined["argument"]:
        if len(tokeniser(_)) <= 8:
            continue

        query = ", ".join(i for i in adu["kp"])
        print(query)

        # TODOs: Add title field for all ES indices to enable multi-field search
        search = [(i["_source"]["document"]["source"], i["_source"]["document"]["text"]) for i in db.search(query_=query, k=retrieve_len)]

        evidence = [i[1] for i in search]
        source = [i[0] for i in search]

        ev_kp = extract_keyphrase(evidence)
        retrieved.append({"passages": evidence, "kp": [i[0] for i in ev_kp], "source": source})

    yield ({
        "id": id_,
        "argument": mined["argument"],
        "retrieved": [i for i in retrieved]
    })

step = max(int(len(unique) / 10), 1)
retrieve_batches = [unique[i:i + step] for i in range(0, len(unique), step)]

retrieved_ev = []
for idx, batch in enumerate(retrieve_batches):
    print('-' * 25 + 'Batch %d/%d' % (idx + 1, len(batch)) + '-' * 25)

    with multiprocessing.Pool(8) as pool:
        with tqdm(total=(len(batch))) as pbar:
            for arg in batch:
                retrieved_ev.append([i for i in search(arg)])
                pbar.update()

  0%|          | 0/5990 [00:00<?, ?it/s]


 t3_4tv5cn
Causal Concept >>  street prostitution Topic >>  None


INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.031s]


['consensual practice', 'occupation', 'prostitution defined', 'prostitution defined consensual', 'sexual activity', 'sexual activity payment', 'payment legal', 'prostitution', 'payment']
consensual practice, occupation, prostitution defined, prostitution defined consensual, sexual activity, sexual activity payment, payment legal, prostitution, payment, street prostitution


INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.129s]


...

INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.151s]


['school collegeb work', 'spoiled lazy kid', 'work parttime', 'high school', 'collegeb work fulltime', 'college', 'collegeb work', 'college spoiled lazy']
school collegeb work, spoiled lazy kid, work parttime, high school, collegeb work fulltime, college, collegeb work, college spoiled lazy


INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.048s]


['real world painful', 'world painful', 'job real world', 'painful humbling', 'job', 'world painful humbling', 'real world']
real world painful, world painful, job real world, painful humbling, job, world painful humbling, real world


IndexError: list index out of range

In [286]:
print(len(retrieved_ev))
retrieved_ev[0]

100


{'id': 't3_30oi71',
 'argument': [{'sentence': 'Section I Why is Basic Income Increasingly Popular?',
   'kp': ['basic income', 'section', 'basic income increasingly'],
   'stance': 'NEUTRAL'},
  {'sentence': 'Basic income is a policy that has broad support from both the progressive left and libertarian right.',
   'kp': ['basic income',
    'libertarian right',
    'broad support',
    'basic income'],
   'stance': 'PRO'},
  {'sentence': 'Centerleft economists including Paul Krugman have endorsed the scheme for various reasons.',
   'kp': ['centerleft economists',
    'paul krugman',
    'scheme',
    'krugman endorsed scheme'],
   'stance': 'PRO'},
  {'sentence': 'BI also reduces inequality by redistributing income from capital to labor.',
   'kp': ['income', 'inequality', 'bi', 'bi reduces inequality'],
   'stance': 'CON'}],
 'retrieved': [{'passages': ["A few special types of gains and losses are not shown in the income statement but as special items in shareholder equity section o

In [240]:
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import torch
import time
import copy

model = SentenceTransformer('all-MiniLM-L6-v2')
def cosine_similarity_(sentences):
    embeddings = model.encode(sentences, convert_to_tensor=True, show_progress_bar=False)

    cos = torch.nn.CosineSimilarity()
    scores = cos(embeddings[0], embeddings[1:])

    scored = []
    retrieved_sentences = sentences[1:]
    for sent, similarity in zip(retrieved_sentences, scores):
        scored.append((sent, similarity.numpy().item()))

    return scored

def rank_passages(ev, k=3):
    adus = [i["sentence"] for i in ev["argument"]]
    retrieved_passages = [i["passages"] for i in ev["retrieved"]]

    # Merge
    # Output 1 x merged sentences object per ADU sentence, with k=5 collected passages as a list of sentences
    merged_passages = []
    for passages in retrieved_passages:
        merged_sents = []
        # Iterate n x sentences for each k=5 retrieved passages
        for passage in passages:
            # Segment as a list of sentences
            sents = sentences_segment(passage)
            # Add sentences to merged_sentences object
            merged_sents.extend(sents)

        # Store merged sentence object for each ADU
        merged_passages.append(merged_sents)

    rank_retrieved = []
    # Rank n x merged sentences for each 1 x ADU
    for adu, merged in zip(adus, merged_passages):
        scored = []
        sentences = [adu]
        sentences.extend(merged)
        scored = cosine_similarity_(sentences)

        ranked_sents = sorted(scored, key=lambda x: x[1], reverse=True)

        # Select top-k sentences
        ranked_sents = ranked_sents[0:k]

        merged = ", ".join(i[0] for i in ranked_sents)
        merged_kp = extract_keyphrase(merged)
        rank_retrieved.append({"ranked_passages": merged, "kp": merged_kp})

    return rank_retrieved

# TODOs: Join passages and sentence rank
### SCORE COSINE SIMILARITY ###
tic = time.time()
retrieved_ranked = copy.deepcopy(retrieved_ev)
with tqdm(total=(len(retrieved_ev)), position=0, leave=True) as pbar:
    for i in range(0, len(retrieved_ev)):
        retrieved_ranked[i]["retrieved"] = [i for i in rank_passages(retrieved_ev[i])]
        pbar.update()
toc = time.time()

retrieved_ranked

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


  0%|          | 0/10 [00:00<?, ?it/s]

[{'id': 't3_30oi71',
  'argument': [{'sentence': 'Section I Why is Basic Income Increasingly Popular?',
    'kp': ['basic income', 'section', 'basic income increasingly'],
    'stance': 'NEUTRAL'},
   {'sentence': 'Basic income is a policy that has broad support from both the progressive left and libertarian right.',
    'kp': ['basic income',
     'libertarian right',
     'broad support',
     'basic income'],
    'stance': 'PRO'},
   {'sentence': 'Centerleft economists including Paul Krugman have endorsed the scheme for various reasons.',
    'kp': ['centerleft economists',
     'paul krugman',
     'scheme',
     'krugman endorsed scheme'],
    'stance': 'PRO'},
   {'sentence': 'BI also reduces inequality by redistributing income from capital to labor.',
    'kp': ['income', 'inequality', 'bi', 'bi reduces inequality'],
    'stance': 'CON'}],
  'retrieved': [{'ranked_passages': "The organisation Basic Income UK is 'a collective of independent people promoting unconditional basic in

In [241]:
print(toc - tic)
len(retrieved_ranked)

33.27585506439209


10

In [244]:
len(retrieved_ranked)
retrieved_ranked[6]

{'id': 't3_5aceoz',
 'argument': [{'sentence': 'UpdateThanks for all the replies theres been some really useful points and Ill dish out the deltas appropriately.',
   'kp': ['deltas', 'useful points', 'replies', 'deltas appropriately'],
   'stance': 'PRO'},
  {'sentence': 'The most convincing argument Ive heard is that Apple is trying to build a computer for the near future and if it doesnt work for you then you dont have to buy it.',
   'kp': ['apple', 'convincing argument', 'computer', 'apple trying build'],
   'stance': 'PRO'},
  {'sentence': 'USBC is the future so why bother putting anything else in?',
   'kp': ['future', 'anything', 'usbc future'],
   'stance': 'CON'},
  {'sentence': 'USB lets you charge from a battery charger which is an extra convenience even if it comes at the loss of MagSafe so why have a dedicated power socket?',
   'kp': ['dedicated power socket',
    'usb',
    'battery charger',
    'dedicated power socket'],
   'stance': 'PRO'},
  {'sentence': 'Most peopl

In [16]:
# NOTE: Zipping retrieved evidence, args
print(len(retrieved_ranked), len(args[0:sample]))

1 1


In [267]:
fout = open("../data/cmv_rr.jsonl", "w")

args = [json.loads(ln) for ln in open("../data/cmv_processed.jsonl")]
sample = args[0:sample]

# for i, j in zip(retrieved_ranked, sample):
#     # Add counter to the dictionary (implicitly, i)
#     i["counter"] = j["counter"]
#     fout.write(json.dumps(i))
#     fout.write("\n")

In [17]:
### EVALUATE OUTPUT ###
train = [json.loads(ln) for ln in open("../data/cmv_rr.jsonl", "r")]

In [18]:
_ = random.randint(0, len(train))
print(train[_]["argument"], "\n")
print(train[_]["retrieved"], "\n")
print(train[_]["counter"], "\n")

[{'sentence': 'I recently got married and one thing my wife and I have discussed at great lengths is what we should do with our surnames.I have come to the conclusion that we currently do not have a reliable fair gender equal convention for dealing with family names in marriage.Example case Groom Jones and Bride JohnsonTheir options are Patrilineal Mr amp Mrs Jones.', 'kp': ['dealing family names marriage', 'family names marriage', 'convention dealing family names'], 'stance': 'PRO'}, {'sentence': 'ConjugateDoubleBarrel Mr amp Mrs JonesJohnson or vice versa No action Mr Jones amp Mrs JohnsonMatrilineal or Patrilineal naming conventions are inherently unbalanced as they require one partner to lose their name for the others.Double Barrels offer a potential solution to this imbalance but I do not think that this is an adequate compromise.', 'kp': ['conjugatedoublebarrel mr amp mrs', 'partner lose double barrels', 'double barrels'], 'stance': 'CON'}, {'sentence': 'If everyone used this for

In [ ]:
# # TODOs: Compute in Batches
# sample = unique[0:100]
# with multiprocessing.Pool(8) as pool:
#     with tqdm(total=(len(unique))) as pbar:
#         for arg in unique:
#             mined_args.append([i for i in extract_adus(arg)])
#             pbar.update()

In [ ]:
# def retrieved_evidence(mined, retrieve_len=5):
#     """ Retrieves Evidence from Knowledge base, returning a well-formed Retrieved Evidence Object
#     given an input Argument"""
#
#     id_ = mined["id"]
#     print("\n", id_)
#
#     retrieved = []
#     adus = []
#     for _ in mined["argument"]:
#         if len(tokeniser(_)) <= 8:
#             continue
#
#         kp = extract_keyphrase(_)
#         print(kp)
#         adu = {"sentence": _, "kp": [i for i in kp], "stance": sentence_stance(_, kp[0])}
#
#         kp.append(topic) if topic else kp
#         kp.append(concept) if concept else kp
#
#         query = ", ".join(i for i in kp)
#         print(query)
#
#         # TODOs: Add title field for all ES indices to enable multi-field search
#         search = [(i["_source"]["document"]["source"], i["_source"]["document"]["text"]) for i in db.search(query_=query, k=retrieve_len)]
#
#         evidence = [i[1] for i in search]
#         source = [i[0] for i in search]
#
#         ev_kp = extract_keyphrase(evidence)
#
#         retrieved.append({"passages": evidence, "kp": [i[0] for i in ev_kp], "source": source})
#         #retrieved.append({"passages": evidence, "source": source})
#         adus.append(adu)
#
#     return ({
#         "id": id_,
#         "argument": [i for i in adus],
#         "retrieved": [i for i in retrieved]
#     })

In [224]:
# from sentence_transformers import SentenceTransformer, util
# from sklearn.metrics.pairwise import cosine_similarity
# import torch
# import time
#
# model = SentenceTransformer('all-MiniLM-L6-v2')
# def cosine_similarity_(sentences):
#     embeddings = model.encode(sentences, convert_to_tensor=True, show_progress_bar=False)
#
#     cos = torch.nn.CosineSimilarity()
#     scores = cos(embeddings[0], embeddings[1:])
#
#     scored = []
#     retrieved_sentences = sentences[1:]
#     for sent, similarity in zip(retrieved_sentences, scores):
#         scored.append((sent, similarity.numpy().item()))
#
#     return scored
#
# def rank_passages(ev, k=3):
#     adus = [i["sentence"] for i in ev["argument"]]
#     retrieved_passages = [i["passages"] for i in ev["retrieved"]]
#
#     # Merge
#     # Output 1 x merged sentences object per ADU sentence, with k=5 collected passages as a list of sentences
#     merged_passages = []
#     for passages in retrieved_passages:
#         merged_sents = []
#         # Iterate n x sentences for each k=5 retrieved passages
#         for passage in passages:
#             # Segment as a list of sentences
#             sents = sentences_segment(passage)
#             # Add sentences to merged_sentences object
#             merged_sents.extend(sents)
#
#         # Store merged sentence object for each ADU
#         merged_passages.append(merged_sents)
#
#     rank_retrieved = []
#     # Rank n x merged sentences for each 1 x ADU
#     for adu, merged in zip(adus, merged_passages):
#         scored = []
#         sentences = [adu]
#         sentences.extend(merged)
#         scored = cosine_similarity_(sentences)
#
#         ranked_sents = sorted(scored, key=lambda x: x[1], reverse=True)
#
#         # Select top-k sentences
#         ranked_sents = ranked_sents[0:k]
#
#         merged = ", ".join(i[0] for i in ranked_sents)
#         merged_kp = extract_keyphrase(merged)
#         rank_retrieved.append({"ranked_passages": merged, "kp": merged_kp})
#
#     return rank_retrieved
#
# # TODO: Clean text
# # TODO: Collect unique Keyphrases per Argument
# rank_passages(retrieved_ev[0])
# #print(len(test["ranked_passages"][0]))

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


[{'ranked_passages': "The organisation Basic Income UK is 'a collective of independent people promoting unconditional basic income as a progressive social policy for the United Kingdom, and beyond'., Several British academics have been involved in the basic income debate., Her son Brandon Rhys Williams proposed a basic income to a parliamentary committee in 1982, and soon after that in 1984, the Basic Income Research Group, now the Citizen's Basic Income Trust, began to conduct and disseminate research on basic income.",
  'kp': ['organisation basic income uk',
   'basic income',
   'basic income debate',
   'basic income uk']},
 {'ranked_passages': "But far more of the contemporary support for basic income in the United States has come from the left of center, driving by people who see it as a major expansion of support for low-income people, as the late, Al Sheahan argued in his 2012 book, 'the Basic Income Guarantee': 'Your right' 'to economic security'., In the Czech Republic, unco

In [62]:
# Handle duplicates
# def rank_passages(ev, k=3):
#     """ Handles a Retrieved Evidence Object, returning the top-k passages for each ADU """
#     # Per Argument
#     # Index into Retrieved Evidence Object
#     adus = [i for i in ev[0]["argument"]]
#     retrieved = [i for i in ev[0]["retrieved"]]
#
#     #print(len(retrieved), len(adus))
#
#     # Rank k-returned passages for each ADU
#     r_retrieved = []
#     for adu, passage in zip(adus, retrieved):
#         scored = []
#         ranked_ev = []
#         for _, kp in zip(passage["evidence"], passage["kp"]):
#             scored.append((_, kp, cosine_similarity(str(adu), str(_))))
#
#         scored = sorted(scored, key=lambda x: x[2], reverse=True)[0:3]
#         for i, j, k in scored:
#             ranked_ev.append({"evidence": i, "kp": j, "similarity": k})
#
#         r_retrieved.append(ranked_ev)
#
#     return r_retrieved

# 1 Argument x 4 ADUs x 5 Retrieved Passages
# ranked = [i for i in rank_passages(retrieved_ev[3])]
# print(len(ranked))
# print(ranked)

4 4
4
[[{'evidence': 'Many technology experts and technology entrepreneurs have begun endorsing basic income in the 2000s and 2010s. These include Marshal Brain, Sam Altman, James Hughes, Facebook co-founder Chris Hughes, Elon Musk, and Mark Zuckerberg (in his 2017 Harvard commencement speech), and Jeremy Rifkin. The overriding theme among technologists who favor basic income is the belief that automation is creating an increasingly unstable labor market.', 'kp': ('technologists favor basic income', 0.6414), 'similarity': 0.3174712657928467}, {'evidence': "Committee member Lady Rhys-Williams argued that the incomes for adults should be more like a basic income. She was also the first to develop the negative income tax model. Her son Brandon Rhys Williams proposed a basic income to a parliamentary committee in 1982, and soon after that in 1984, the Basic Income Research Group, now the Citizen's Basic Income Trust, began to conduct and disseminate research on basic income.", 'kp': ('will

In [13]:
# from sentence_transformers import SentenceTransformer, util
# import torch
# import time
#
# model = SentenceTransformer('all-MiniLM-L6-v2')
#
# # TODOs: Join passages and sentence rank
# ### SCORE COSINE SIMILARITY ###
# def cosine_similarity(sent_1, sent_2):
#     sentences = [sent_1, sent_2]
#     embeddings = model.encode(sentences, convert_to_tensor=True, show_progress_bar=False)
#
#     cos = torch.nn.CosineSimilarity(dim=0)
#     score = cos(embeddings[0], embeddings[1])
#
#     return score.numpy().item()
#
# ### SCORE TF-KEYWORD OVERLAP ###
# def overlap_score(evidence_kp, adu_kp):
#     score = 0
#     # Split Keyphrase into components, scoring partial units as overlap
#     for i in evidence_kp:
#         for j in i.split():
#             # Ensure string value, to enact .find
#             if ", ".join([i for i in adu_kp]).find(j) != -1: score += 1
#
#             else: continue
#     return score
#
# ### RANK PASSAGES ###
# def score_passages(ev):
#     for _ in range(0, len(ev["argument"])):
#         print(_)
#
# from collections import defaultdict
# def rank_passages(ev, k=2):
#     adus = [i for i in ev["argument"]]
#     retrieved = [i for i in ev["retrieved"]]
#
#     rank_retrieved = []
#     count = 0
#
#     for adu, passages in zip(adus, retrieved):
#         count += 1
#         scored = []
#
#         # 5 passages
#         for passage in passages["passages"]:
#             score = cosine_similarity(str(adu), str(passage))
#             scored.append((passage, score))
#
#         ranked_passages = sorted(scored, key=lambda x: x[1], reverse=True)
#         ranked_passages = ranked_passages[0:k]
#
#         merged = ", ".join(i[0] for i in ranked_passages)
#         merged_kp = extract_keyphrase(merged)
#         rank_retrieved.append({"ranked_passages": merged, "kp": merged_kp})
#
#     return rank_retrieved
#
# import copy
# ### UPDATE RETRIEVED OBJECT ###
# # for i in range(0, len(retrieved_ev)):
# #     retrieved_ranked[i]["retrieved"] = [i for i in rank_passages(retrieved_ev[i])]
#
# tic = time.time()
# retrieved_ranked = copy.deepcopy(retrieved_ev)
# with tqdm(total=(len(retrieved_ev)), position=0, leave=True) as pbar:
#     for i in range(0, len(retrieved_ev)):
#         retrieved_ranked[i]["retrieved"] = [i for i in rank_passages(retrieved_ev[i])]
#     pbar.update()
#
# toc = time.time()
# # duration = toc - tic
#
# retrieved_ranked

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


  0%|          | 0/10 [00:00<?, ?it/s]

[{'id': 't3_30oi71',
  'argument': [{'sentence': 'Section I Why is Basic Income Increasingly Popular?',
    'kp': ['basic income increasingly',
     'basic income',
     'section basic income'],
    'stance': 'NEUTRAL'},
   {'sentence': 'Basic income is a policy that has broad support from both the progressive left and libertarian right.',
    'kp': ['basic income', 'basic income policy', 'income policy'],
    'stance': 'PRO'},
   {'sentence': 'Centerleft economists including Paul Krugman have endorsed the scheme for various reasons.',
    'kp': ['krugman endorsed scheme',
     'centerleft economists including',
     'centerleft economists'],
    'stance': 'PRO'},
   {'sentence': 'BI also reduces inequality by redistributing income from capital to labor.',
    'kp': ['bi reduces inequality',
     'inequality redistributing income',
     'reduces inequality redistributing'],
    'stance': 'CON'}],
  'retrieved': [{'ranked_passages': "Several British academics have been involved in the b

In [202]:
# Fix KW extraction
# Fix Duplicates
# def rank_passages(ev, k=2):
#     adus = [i for i in ev["argument"]]
#     retrieved = [i for i in ev["retrieved"]]
#
#     rank_retrieved = []
#     count = 0
#
#     for adu, passages in zip(adus, retrieved):
#         count += 1
#         scored = []
#
#         # 5 passages
#         for passage in passages["passages"]:
#             score = cosine_similarity(str(adu), str(passage))
#             scored.append((passage, score))
#
#         ranked_passages = sorted(scored, key=lambda x: x[1], reverse=True)
#         ranked_passages = ranked_passages[0:k]
#
#         merged = ", ".join(i[0] for i in ranked_passages)
#         merged_kp = extract_keyphrase(merged)
#         rank_retrieved.append({"ranked_passages": merged, "kp": merged_kp})
#
#     return rank_retrieved
#
# ev = retrieved_ev[0]
# rank_passages(ev)

[{'ranked_passages': "Several British academics have been involved in the basic income debate. Among them the following:Organisations. The organisation Basic Income UK is 'a collective of independent people promoting unconditional basic income as a progressive social policy for the United Kingdom, and beyond'., Basic Income. Delaney opposes implementing a basic income (also known as a universal basic income). Minimum wage.",
  'kp': ['organisation basic income uk',
   'basic income uk',
   'united kingdom basic income']},
 {'ranked_passages': "Several British academics have been involved in the basic income debate. Among them the following:Organisations. The organisation Basic Income UK is 'a collective of independent people promoting unconditional basic income as a progressive social policy for the United Kingdom, and beyond'., A basic income is defined in the report as a policy that guarantees all members of a society a minimum amount of income. One type of basic income considered is

In [458]:
# def fuck_you():
#     print("fuck you")
#
# fuck_you()
#
# def rank_(ev):
#     # Index into Retrieved Evidence Object
#     ev = ev[0]
#     adus = [i for i in ev[0]["argument"]]
#     retrieved = [i for i in ev[0]["retrieved"]]
#     k = 3
#     print("hello")
#     # # Rank k-returned passages for each ADU
#     # count = 0
#     # r_retrieved = []
#     # for adu, passage in zip(adus, retrieved):
#     #     count += 1
#     #     ranked_passages = []
#     #     for _ in passage["evidence"]:
#     #         print(_)
#     #         ranked_passages.append((_, cosine_similarity(adu, _)))
#     #         r_retrieved.append({"evidence": i, "similarity": k} for i, k in sorted(ranked_passages, key=lambda x: x[1], reverse=True)[0:k])
#     #         print(r_retrieved)
#
#     # return {
#     #     "r_retrieved": r_retrieved
#     # }
#
# #print(rank_(retrieved_ev[0:1]))

In [116]:
# from multiprocessing.pool import ThreadPool as Pool
# from yake import KeywordExtractor
# import tqdm.notebook as tqdm
# import time
# from summa import keywords
# from tqdm import tqdm
#
# ### PASSAGE RANKING; KEYWORD OVERLAP ###
# kw_extractor = KeywordExtractor(lan="en", n=3, top=5)
#
# # TODOs: For each ADU, Rank Merged Evidence using Keyword Overlap and Filter for Contrasting Stance
# # TODOs: Handel Multiple Keywords
#
# def overlap_score(evidence_kp, adu_kp):
#     score = 0
#     # TODOs: Robust 'None' handeling
#     if adu_kp == None:
#         return score
#     # Split Keyphrase into components, scoring partial units as overlap
#     else:
#         for i in evidence_kp:
#             for j in i.split():
#                 # Ensure string value, to enact .find
#                 if ", ".join([i for i in adu_kp]).find(j) != -1: score += 1
#
#                 else: continue
#
#     return score
#
# def calculate_overlap(merged_ev, adu_kp):
#
#     for ev_unit in sentences_segment(merged_ev):
#         toks = tokeniser(ev_unit)
#         kp_overlap = 0
#
#         if len(toks) <= 8: continue
#
#         #ev_unit_kp = [i for i in keywords.keywords(ev_unit).split("\n")]
#         ev_unit_kp = [i[0] for i in kw_extractor.extract_keywords(ev_unit)]
#
#         if ev_unit_kp:
#             kp_overlap = overlap_score(evidence_kp=ev_unit_kp, adu_kp=adu_kp)
#
#         else: ev_unit_kp = None
#         yield ev_unit, ev_unit_kp, kp_overlap
#
# # pool = Pool(8)
# ### RANK PASSAGES ###
# def score_passages(ev_):
#     adu = ev_[0]["argument_discourse_unit"]
#     adu_stance = ev_[0]["adu_stance"]
#     merged_ev = ev_[0]["merged_evidence"]
#     adu_kp = ev_[0]["adu_keyphrases"]
#
#     ### CALCULATE OVERLAP ###
#     for ev_unit, ev_unit_kp, kp_overlap in calculate_overlap(merged_ev, adu_kp):
#         target = adu_kp[0]
#
#         compared_stace = compare_stance(ev_unit, target)
#         if compared_stace != adu_stance:
#             yield {
#                 "adu": adu,
#                 "adu_kp": adu_kp,
#                 "evidence_unit": ev_unit,
#                 "evidence_kps": ev_unit_kp,
#                 "overlap": kp_overlap,
#                 "evidence_stance": compare_stance(ev_unit, target),
#                 "adu_stance": adu_stance
#             }
#
#         else: continue
#
# ### SCORED EVIDENCE ###
# def score_evidence(retrieved_evidence):
#     for ev_ in retrieved_ev:
#         yield [i for i in score_passages(ev_)]
#
# ### RANKED EVIDENCE ###
# def rank_filter_counter_evidence(retireved_evidence, k=3):
#     with tqdm(total=(len(retrieved_ev))) as pbar:
#         for i in score_evidence(retrieved_ev):
#             yield sorted(i, key=lambda y: y["overlap"], reverse=True)[0:k]
#
#             pbar.update()
#
#
# ### SELECT TOP-K COUNTER-EVIDENCE ###
# tic = time.time()
# ranked_sorted_evidence = [i for i in rank_filter_counter_evidence(retrieved_ev)]
# ranked_sorted_evidence
# toc = time.time()
#
# print(toc - tic)
# # TIME 1:20M

100%|██████████| 100/100 [00:30<00:00,  3.23it/s]

30.97145128250122


In [250]:
# idx = 2
# for ln in retrieved_ev:
#     r = ln[0]
#     for _ in range(0, len(r["argument"])):
#         print(r["argument"][_]["sentence"])
#         print(r["argument"][_]["kp"])
#         print("")
#         print(r["retrieved"][_]["evidence"])
#         print(r["retrieved"][_]["kp"])

#"counter": {"counter": arg["counter"]["counter"], "counter_kp": arg["counter"]["counter_keyphrases"]}
# "argument_discourse_unit": adu,
# "query": query,
# "adu_keyphrases": [i for i in kp],
# "adu_stance": sentence_stance(adu, kp),
# "merged_evidence": ", ".join(ln for ln in evidence)
# "retrieved_documents_titles": titles,
# "retrieved_evidence": evidence,

Section I Why is Basic Income Increasingly Popular?
['basic income increasingly popular', 'basic income increasingly', 'section basic income increasingly']

Brazil. Minimum income has been increasingly accepted by the Brazilian government. In 2004, President Lula da Silva signed into law a bill to establish a universal basic income. Committee member Lady Rhys-Williams argued that the incomes for adults should be more like a basic income. She was also the first to develop the negative income tax model. Her son Brandon Rhys Williams proposed a basic income to a parliamentary committee in 1982, and soon after that in 1984, the Basic Income Research Group, now the Citizen's Basic Income Trust, began to conduct and disseminate research on basic income. Many technology experts and technology entrepreneurs have begun endorsing basic income in the 2000s and 2010s. These include Marshal Brain, Sam Altman, James Hughes, Facebook co-founder Chris Hughes, Elon Musk, and Mark Zuckerberg (in his 201

In [ ]:
# # TODOs: Speed-up, Parrelleise, Yield
# def overlap_score(evidence_kp, adu_kp):
#     score = 0

#     # Split Keyphrase into components, scoring partial units as overlap
#     for i in evidence_kp:
#         for j in i.split():
#             # Ensure string value, to enact .find
#             if " ".join(adu_kp).find(j) != -1: score += 1

#             else: continue

#     return score

# ev_units = evidence
# adu_kp = extract_keyphrase(adu)

# adu_ev_overlap = []

# kp_1 = ['sex', 'relationship', 'opportunity']
# kp_2 = ['better sex']

# overlap_score(kp_2, kp_1)

# for ev_unit in evidence:
#     #print(ev_unit)
#     toks = tokeniser(ev_unit)

#     # Exprimental Value
#     if len(toks) <= 8:
#         continue

#     ev_unit_kp = extract_keyphrase(ev_unit)
#     kp_overlap = overlap_score(evidence_kp=ev_unit_kp, adu_kp=adu_kp)

#     adu_ev_overlap.append({
#         "adu": adu,
#         "adu_kp": adu_kp,
#         "ev_unit": ev_unit,
#         "ev_unit_kp": ev_unit_kp,
#         "kp_overlap": kp_overlap

#         })

# adu_ev_overlap

In [ ]:
# ### OVERLAP RANKED EVIDENCE ###

# adu_ev_overlap.sort(key=lambda y: y["kp_overlap"], reverse=True)
# adu_ev_overlap

# ### FILTER IRRELEVANT EVIDENCE ###
# overlapping = [i for i in adu_ev_overlap if i["kp_overlap"] !=0]

# len(adu_ev_overlap), len(overlapping)
# overlapping


In [15]:
# Stance Test
# adu = 'I cant remember the topic that spurred this discussion but a friend and I were debating whether manmade things were natural.'
# ev_unit = 'In this essay, Mill argues the idea that the morality of an action can be judged by whether it is natural or unnatural.'
# target = 'natural things'
#
# stance = compare_stance(ev_unit, target)
# stance

'PRO'

In [ ]:
# ### ASSERT SAME STANCE ###
# from detection.stance_classifier import sentence_stance, compare_stance
#
# # TODOs: Ensure KPs Extracts are constrained to 1 unit
# opposing_stance = []
# for i in overlapping:
#     adu = i["adu"]
#     target = " ".join(i for i in i["adu_kp"])
#     ev_unit = i["ev_unit"]
#
#     ev_stance = compare_stance(ev_unit, ev_unit, target)
#     adu_stance = sentence_stance(adu, target)
#
#     if ev_stance != adu_stance:
#         opposing_stance.append((ev_unit, ev_stance, adu_stance))
#
#     else: continue
#
# opposing_stance

In [ ]:
### RANKING ###

# TODOs: Speed-up, Parrelleise, Yield
# ev_units = evidence
# adu_kp = extract_keyphrase(adu)

# adu_ev_overlap = []

# kp_1 = ['sex', 'relationship', 'opportunity'] 
# kp_2 = ['better sex']

# overlap_score(kp_2, kp_1)

# for ev_unit in evidence:
#     #print(ev_unit)
#     toks = tokeniser(ev_unit)

#     # Exprimental Value
#     if len(toks) <= 8:
#         continue
    
#     ev_unit_kp = extract_keyphrase(ev_unit)
#     kp_overlap = overlap_score(evidence_kp=ev_unit_kp, adu_kp=adu_kp)
    
#     adu_ev_overlap.append({
#         "adu": adu, 
#         "adu_kp": adu_kp,
#         "ev_unit": ev_unit,
#         "ev_unit_kp": ev_unit_kp, 
#         "kp_overlap": kp_overlap
        
#         })
        
# adu_ev_overlap


#rank_passages(retrieved_ev)

In [ ]:
# import spacy
# from spacy.matcher import PhraseMatcher
# from fuzzywuzzy import fuzz, process

# # TODOs: Package as a Module
# # TODOs: Handle Negation (Polarity shifters)
# # TODOs: Review Unsuperived Approach; Consider adveanced patterns and common-sence knowledge

# nlp = spacy.load("en_core_web_sm")

# sentence = "I hate abortion rights. Abortions should be banned."
# sentence_2 = "I like abortion rights. I belive we should keep them."
# sentence_3 = "I hate tennis. People should play tennis more often"

# ### STANCE SCORING ###

# # TODOs: https://www.cs.uic.edu/~liub/FBS/opinion-mining-final-WSDM.pdf 
# # TODOs: Pattern based Negation
# # TODOs: Semantic Orientation of an opinion (Claim)
# # TODOs:Group synonyms of 'features', 'targets'

# phrase_matcher = PhraseMatcher(nlp.vocab)

# ### SENTIMENT LEXICONS ###
# pos = [w.replace("\n", "") for w in open("../../data/lexicon/positive_lex.txt")]
# neg = [w.replace("\n", "") for w in open("../../data/lexicon/negative_lex.txt")]
# polarity_shifters = [w.replace("\n", "") for w in open("../../data/lexicon/shifter_lexicon.txt")]

# ### STANCE: ASPECT-SEMANTIC ORIENTATION ###
# def extract_aspect(sentence, n_gram):
#     aspects = extract_keyphrase(str(sentence))[0]

#     return nlp(aspects)

# def index_aspect(phrase, aspect, sentence):    
#     patterns = [nlp(aspect)]
#     phrase_matcher.add(phrase, None, *patterns)

#     start = 0
#     stop = 0

#     matched_phrases = phrase_matcher(sentence)
#     for i in matched_phrases:
#         _, start, stop = i
        
#     return start, stop

# # TODOs: Implement Polarity Shifters, Simple
# # TODOs: Implement Polarity Shifters, Complex, Verb Patterns
# def stance_score(start, stop, sentence):
#     pos_score = 0.0
#     neg_score = 0.0

#     score = 0
#     for idx, tok in enumerate(sentence):
#         if idx == start or idx == stop:
#             continue

#         # TODOs: Implement Polarity Shift
#         # TODOs: Experiement with descriptive term + keyphrase aspects
#         # TODOs: ABSA https://www.kaggle.com/code/phiitm/aspect-based-sentiment-analysis
#         # Use external libaray: Textblob
        
#         k = 8
#         # Negation Rules
#         shifted_tok = None
#         shifted_toks = []

#         if (tok.dep_ == "neg") or (tok.dep_ in polarity_shifters):
#             #Shift to Negative
#             if idx <= k:
#                 if idx < start: neg_score += 1/(start - idx)
#                 else: neg_score += 1/(idx - stop)**0.5

#             if shifted_tok != None and shifted_tok in neg:
#                 print(shifted_tok.text)
#                 # Shift to Positive
#                 if idx < start: pos_score += 1/(start - idx)
#                 elif idx > start: pos_score += 1/(idx - stop)**0.5
#                 else: continue

#         # Aspect Sentement Orientation
#         if tok.text in pos:
#             if tok in shifted_toks:
#                 continue
            
#             if idx < start: pos_score += 1/(start - idx)
#             else: pos_score += 1/(idx - stop)**0.5

#         if tok.text in neg:
#             if tok in shifted_toks:
#                 continue

#             if idx <= start: neg_score += 1/(start - idx)
#             else: neg_score += 1/(idx - stop)**0.5
    
#     score = pos_score - neg_score /(pos_score + neg_score + 1)

#     return score

# def overlap_score(evidence_kp, adu_kp):
#     score = 0
    
#     # Split Keyphrase into components, scoring partial units as overlap
#     for i in evidence_kp:
#         for j in i.split():
#             # Ensure string value, to enact .find
#             if " ".join(adu_kp).find(j) != -1: 
#                 score += 1
#                 token = j
            
#             else: continue
    
#     return score

# def get_overlapping_token(evidence_kp, adu_kp):
#     for i in evidence_kp:
#         overlap_tokens = []
#         for j in i.split():
#             if " ".join(adu_kp).find(j) != -1: 
#                 overlap_tokens.append(j) 
            
#         return " ".join(i for i in overlap_tokens)

# def sentence_stance(sentence, aspect):
#     sentence = nlp(sentence)

#     start, stop = index_aspect("aspects", aspect, sentence)
#     score = stance_score(start, stop, sentence)

#     # Add Neutral
#     #stance = {"claim": sentence, "stance": "PRO", "aspect": aspect} if score > 0 else {"claim": sentence, "stance": "CON", "aspect": aspect}
    
#     return "PRO" if score > 0 else "CON"

# def fuzzy_match(target, evidence_unit):

#     overlapping_aspect = process.extractOne(target, ev.split())[0]
#     score = overlapping_aspect[1]

#     overlapping_aspect = nlp(re.sub(r'[^\w]', ' ', overlapping_aspect))

#     return overlapping_aspect, score

# def compare_stance(ev_unit, evidence_aspect, adu_target):
#     # Note: Already identified mathcing or partially matching Aspects. 

#     # Get the overlapping evidence aspect-target.
#     overlapping_target, score = fuzzy_match(target=adu_aspect, evidence_unit=ev)
    
#     # Get position of the overlapping_target
#     start, stop = index_aspect("OVERLAP", nlp(overlapping_target), nlp(ev_unit))

#     # Assert Stance towards evidence aspect
#     score = stance_score(start, stop, nlp(ev_unit))
    
#     return "PRO" if score > 0 else "CON"

# ev = "These simple ideas and techniques could help both you and your lover enjoy sex. 1 / 10 Getty Images/Caiaimage Think beyond the thrust."
# ev_aspect = "sex", "relationship", "opportunity"

# adu = 'Hello! Let me preface by saying I dont believe there is a better sex.'
# adu_aspect = "better sex"

# print(sentence_stance("The mutual trust and understanding you share with your partner will lead to better sex, but that's not the only reason sex can be better when you're not in a relationship.", adu_aspect))
# print(compare_stance(ev, ev_aspect, adu_aspect))


In [ ]:
# from spacy.matcher import DependencyMatcher, Matcher
# matcher = Matcher(vocab=nlp.vocab)
# matcher

# # Matching Rule: Pronouns with Verbs that follow them
# aspect = "better sex"
# patterns = [
#     [{"DEP": "neg"}, {"LOWER": aspect}],
#     [{"DEP": "neg"}, {"POS": "ADJ"}, {"LOWER": aspect}],
#     [{"POS": "VERB"}, {"POS": "ADJ"}, {"LOWER": aspect}],
#     [{"LOWER": aspect.lower()}]
# ]

# test = nlp("Hello! Let me preface by saying I dont believe there is a not better sex.")
# test_2 = nlp("These simple ideas and techniques could help both you and your lover enjoy better sex.")

# matcher.add("test", patterns=patterns)
# result = matcher(test_2, as_spans=True)

# result

# # for tok in test:
# #     print(tok.i, tok, tok.pos_, tok.dep_, tok.head.i, sep="\t")

In [ ]:
### TARGETED RETRIEVAL: ATTACKING PEMISES ###

# from BERT_adu_classifier import predict

# premises = []
# for sent in sentences:
#     prediction = predict(sent)
    
#     if prediction == "premise":
#         premises.append(sent)